In [1]:
!pip install langchain-core langchain-groq langserve python-dotenv langchain_chroma bs4


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 9.9 MB/s eta 0:00

In [2]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.6 MB/s eta 0:00:00


In [29]:
from google.colab import userdata
hug_face_key = userdata.get('HUGGINGfACEKEY')
groq_key = userdata.get('Groq_key')
Lang_key = userdata.get('Lanchain_key')

In [30]:
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [13]:
!pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [31]:
llm = ChatGroq(groq_api_key=groq_key,model_name="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7a59f05a07f0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7a59f05a1c90>, model_name='Llama3-8b-8192', groq_api_key=SecretStr('**********'))

In [32]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

## 1.Load,chunk and index the contents of the blog to create a retreiver.


In [33]:
import bs4
loader = WebBaseLoader(
    web_paths =("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only = bs4.SoupStrainer(
            class_ = ("post-content","post-title","post-header")
        )
    ),

)

In [34]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [35]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
retreiver = vectorstore.as_retriever()
retreiver

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7a59f05a3550>)

In [36]:
## Prompt Template
system_prompt = (
    "You are an AI assistant providing helpful advice. "
    "You are given the following extracted parts of a long document and a question. "
    "Provide a conversational answer based on the context provided. "
    "If you don't know the answer, just say that you don't know. "
    "----------------"
    "CONTEXT: {context}"
    "----------------"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

In [37]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retreiver,question_answer_chain)

In [38]:
response = rag_chain.invoke({"input": "what is Self-Reflection?"})
response['answer']

"Based on the context provided, Self-Reflection is a mechanism in the Reflexion framework that helps an agent learn from its mistakes. It's created by showing the agent two-shot examples, which are pairs of a failed trajectory and an ideal reflection for guiding future changes in the plan.\n\nIn essence, Self-Reflection allows the agent to reflect on its past actions and identify what went wrong, so it can adjust its behavior accordingly. This process helps the agent to refine its planning and decision-making over time."

In [39]:
response1 = rag_chain.invoke({"input": "How do we acheive it?"})
response1['answer']

"Based on the context provided, it seems that you're referring to achieving the Reflexion framework, which is a system that uses self-reflection to improve the planning and decision-making of an agent.\n\nTo achieve this, the Reflexion framework uses a combination of techniques, including:\n\n1. Showing two-shot examples to the LLM (Large Language Model) to create self-reflection. Each example is a pair of (failed trajectory, ideal reflection for guiding future changes in the plan).\n2. Adding up to three reflections into the agent's working memory to be used as context for querying the LLM.\n3. Using the heuristic function to determine when a trajectory is inefficient or contains hallucination and should be stopped.\n\nAdditionally, the system uses a 4-stage process, which includes task planning, task parsing, planning, and execution. The LLM works as the brain and parses the user requests into multiple tasks, and uses few-shot examples to guide the planning process.\n\nIt's worth not

## Adding chat history

In [42]:
from langchain.chains import create_history_aware_retriever
from langchain.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given the following conversation and a follow up question, "
    "rephrase the follow up question to be a standalone question, in its original language. "
    "Given a chat history and the latest user question"
    "Which might reference context in the chat history,"
    "formulate a standlone quesiton which can be understood"
    "without the chat history.Do not answer the quesition"
)

contextulaize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ]
)

In [43]:
history_aware_retreiver = create_history_aware_retriever(llm,retreiver,contextulaize_q_prompt)

In [44]:
history_aware_retreiver

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7a59f05a3550>))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language. Given a chat history and the latest user questionWhich might reference context in the

In [45]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(history_aware_retreiver,question_answer_chain)

In [46]:
from langchain_core.messages import AIMessage,HumanMessage
Chat_history = []
question = "what is self-reflection?"
Chat_history.append(HumanMessage(content=question))
response = rag_chain.invoke({"input": question, "chat_history": Chat_history})
response['answer']

"Based on the context you provided, self-reflection is a mechanism in the Reflexion framework that helps an agent learn from its mistakes and improve its planning. It's created by showing the agent two-shot examples, which are pairs of a failed trajectory and an ideal reflection for guiding future changes in the plan. These reflections are then added to the agent's working memory, up to a maximum of three, to be used as context for querying a Large Language Model (LLM). In other words, self-reflection allows the agent to reflect on its past mistakes, learn from them, and adjust its behavior accordingly."

In [47]:
question1 = "Tell more about it?"
Chat_history.append(HumanMessage(content=question1))
response = rag_chain.invoke({"input": question, "chat_history": Chat_history})
response['answer']

'Based on the provided context, self-reflection in the Reflexion framework is created by showing two-shot examples to Large Language Model (LLM) and each example is a pair of (failed trajectory, ideal reflection for guiding future changes in the plan). This means that self-reflection is a process where the agent reflects on its past failures or inefficient planning by comparing it to an ideal reflection, with the goal of making adjustments to improve its future planning.'

In [48]:
Chat_history

[HumanMessage(content='what is self-reflection?'),
 HumanMessage(content='Tell more about it?')]